In [1]:
import sys
import pandas as pd
import altair as alt

def read_data(path):
    data = pd.read_csv(path)
    return data.fillna(0)

def squash(group):
    x = group.iloc[0]
    x["Files Changed"] = int(group["Files Changed"].sum())
    x["Insertions"] = int(group["Insertions"].sum())
    x["Deletions"] = int(group["Deletions"].sum())
    x["Count"] = len(group)                                                                                                                                                        
    return x

def compute_top_committers(data, topnr):
    data = data.groupby("Name").apply(squash)
    # TODO: grab formula from Zak for ranking
    return data.sort_values(by=['Count'], ascending=False).head(n=topnr)

def compute_organisational_commits(data):
    data = data.groupby("Organization").apply(squash)
    return data

def plot_result(data):
    pass

# **Top 10 Individual Commiters in the last year**

In [2]:
path = "./data/lastyear_commits.csv"
data = read_data(path)

In [3]:
top = compute_top_committers(data, 10)

In [4]:
pd.set_option('display.expand_frame_repr', False)
top[['Count', 'Files Changed', 'Insertions', 'Deletions']]

,Count,Files Changed,Insertions,Deletions
Name,,,,
Marko Mäkelä,290,1698,25158,44018
Jan Lindström,94,344,3566,8186
Sergei Golubchik,45,148,14190,2009
Michael Widenius,45,154,1901,503
Daniel Black,36,102,2222,1031
Vladislav Vaintroub,33,93,3565,5256
Vicențiu Ciorbaru,28,375,3180,2207
Eugene Kosov,25,106,1711,1356
Kentoku Shiba,24,166,13368,4173


# **Top 10 Individual Commiters all time**

In [5]:
path = "./data/alltime_commits.csv"
data = read_data(path)

In [6]:
top = compute_top_committers(data, 10)

In [7]:
top[['Count', 'Files Changed', 'Insertions', 'Deletions']]

,Count,Files Changed,Insertions,Deletions
Name,,,,
Marko Mäkelä,2010,13601,274289,657697
Sergei Golubchik,1743,16773,1139786,918222
Vladislav Vaintroub,910,3470,84706,56866
Vasil Dimov,757,4915,31816,73951
Michael Widenius,647,8696,255548,55130
Alexander Barkov,644,4050,965303,148402
Sergey Vojtovich,600,2205,31205,55296
Sergey Petrunya,539,2275,134121,24535
Georgi Kodinov,530,2028,29115,13109


# **Number of all-time commits per top 30 contributors**

In [8]:
top30 = compute_top_committers(data, 30)
alt.Chart(data=top30, title=f"Number of commits").mark_bar().encode(
    x=alt.X('Name', type='nominal', sort=top30['Name'].values.tolist()),
    y='Count',
    color='Name')

alt.Chart(...)

# **Organisational Commits last year**

In [9]:
path = "./data/lastyear_commits.csv"
data = read_data(path)

In [10]:
org_commits = compute_organisational_commits(data)

In [11]:
org_commits[['Count', 'Files Changed', 'Insertions', 'Deletions']]

,Count,Files Changed,Insertions,Deletions
Organization,,,,
Codership,22,76,2385,205
IBM,3,8,42,211
MariaDB Corporation,608,2973,66182,68584
MariaDB Corporation and MariaDB Foundation,45,154,1901,503
MariaDB Foundation,45,415,3695,2350
No Organization,126,489,20643,6397
Oracle,4,4,27,21
Tempesta,55,215,3051,1813


# **Organisational Commits all time**

In [12]:
path = "./data/alltime_commits.csv"
data = read_data(path)

In [13]:
org_commits = compute_organisational_commits(data)

In [14]:
org_commits[['Count', 'Files Changed', 'Insertions', 'Deletions']].sort_values(by=['Count'], ascending=False).head(15)

,Count,Files Changed,Insertions,Deletions
Organization,,,,
MariaDB Corporation,5503,39246,2101919,1770016
Oracle,5042,40199,976455,939125
Sun,4457,26488,1403265,1482953
MySQL,1625,10687,2776978,198930
Monty Program,1192,7216,379919,55558
No Organization,1133,16107,401539,174146
Tokutek,990,5896,4096817,1503015
MariaDB Foundation,371,3029,36264,21309
Codership,331,1113,30343,16535


# **Top 30 PR authors in the last year**

In [18]:
prs = pd.read_csv('./data/lastyear_github.csv')

def count(group):
    x = group.iloc[0]
    x['Count'] = len(group)
    return x
prs_count = prs.groupby('author').apply(count).sort_values(by=['Count'], ascending=False).head(30)

alt.Chart(data=prs_count, title=f'Number of PRs per contributor in the last year').mark_bar().encode(
    x=alt.X('author', sort='-y'),
    y='Count',
    color='author')

alt.Chart(...)

# **Top 30 PR authors alltime**

In [19]:
prs = pd.read_csv('./data/alltime_github.csv')
prs_count = prs.groupby('author').apply(count).sort_values(by=['Count'], ascending=False).head(30)

alt.Chart(data=prs_count, title=f'Number of PRs per contributor alltime').mark_bar().encode(
    x=alt.X('author', sort='-y'),
    y='Count',
    color='author')

alt.Chart(...)

# **Contributor Frustration Metric**

In [20]:
prs = pd.read_csv('./data/alltime_github.csv')

open_prs = prs.loc[prs['state'] == 'OPEN']
prs_count = open_prs.groupby('author').apply(count)

alt.Chart(data=prs_count, title=f'Number of PRs still in OPEN state per contributor').mark_bar().encode(
    x=alt.X('author', sort='-y'),
    y='Count',
    color='author')

alt.Chart(...)

# **Response Time for PRs**

In [23]:
def get_response_time(row):
    closed = pd.to_datetime(row['closedAt'])
    created = pd.to_datetime(row['createdAt'])
    response_time = (closed - created).total_seconds()
    row['Response Time'] = response_time / (3600 * 24)
    row['createdAt'] = created
    row['closedAt'] = closed
    return row

prs = pd.read_csv('./data/alltime_github.csv')
closed_prs = prs.loc[prs['state'].isin(['CLOSED', 'MERGED'])]
closed_prs = closed_prs.apply(get_response_time, axis=1)

In [24]:
print("Average response time for a PR:", closed_prs['Response Time'].mean(), "days")

Average response time for a PR: 96.22749612744803 days


In [26]:
alt.Chart(closed_prs).mark_circle(size=60).encode(
    x='closedAt',
    y='Response Time',
    tooltip=['author', 'Response Time', 'title']
).interactive()

alt.Chart(...)

# **Contributor Frustration Metric 2 (Counting Time)**

In [83]:
prs = prs.apply(get_response_time, axis=1)

def compute_total_waited_time(group):
    x = group.iloc[0]
    x['Total Waited Time'] = group['Response Time'].sum()
    return x

prs_time = prs.groupby('author')\
              .apply(compute_total_waited_time)\
              .sort_values(by=['Total Waited Time'], ascending=False)\
              .head(30)

alt.Chart(data=prs_time, title=f'Total time a contributor waited for its PRs to be merged').mark_bar().encode(
    x=alt.X('author', sort='-y'),
    y=alt.Y('Total Waited Time', title='Total Waited Time (days)'),
    color='author')

alt.Chart(...)

# **PR Backlog Size at the end of each year**

In [84]:
years = [2014, 2015, 2016, 2017, 2018, 2019, 2020]
backlogs = [read_data(f'./data/pr_backlog/{year}_pr_backlog.csv') for year in years]

In [85]:
no_prs_per_year = [len(backlogs[i]) for i in range(len(years))]
source = pd.DataFrame({
    'year': years,
    'backlog': no_prs_per_year
})

In [86]:
alt.Chart(data=source, title=f'The backlog size at the end of the year'
    ).mark_line(point=True).encode(
        x='year:O',
        y='backlog'
    ).properties(
        width=400,
        height=300
    )

alt.Chart(...)

# **Number of Pull Requests opened each year**

In [88]:
def year_pr_was_opened(row):
    created = pd.to_datetime(row['createdAt'])
    row['year'] = created.year
    return row
    

In [111]:
prs = prs.apply(year_pr_was_opened, axis=1)
prs_per_year = prs.groupby('year').apply(count)

In [110]:
alt.Chart(data=prs_per_year, title=f'Number of PRs opened each year'
    ).mark_bar().encode(
        x=alt.X('year:O'),
        y=alt.Y('Count', title='Number of PRs')
    ).properties(
        width=400,
        height=300
    )

alt.Chart(...)